In [12]:
%run set_theme.ipynb

In [ ]:
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

In [ ]:
df = pd.read_parquet('../data/SO_2014_2022.pq')

df = df[(df['Salary'] > 0) & (df['Salary'] < 250000)]

df.head()

In [ ]:
# Show the top-n countries in the plot.
TOP_N_COUNTRIES = 10

# Take the top-n countries that have the most developers.
countries = df[['Country', 'RespondentType']] \
    .query('RespondentType == "dev"') \
    .query('Country != "Brazil"') \
    .query('Country != "India"') \
    .query('Country != "Russia"') \
    .query('Country != "Sweden"') \
    .groupby('Country', as_index=False) \
    .count() \
    .sort_values('RespondentType', ascending=False) \
    .head(TOP_N_COUNTRIES) \
    ['Country'] \
    .tolist()

countries_df = df[df['Country'].isin(countries)]
    
# Calculate the salary gap in percentage.
salary_men_df = countries_df.query('Gender == "male"') \
    .groupby(['Year', 'Country']) \
    .agg({'Salary': 'mean' }) \
    .reset_index() \
    .rename(columns={'Salary': 'SalaryMen'})

salary_women_df = countries_df.query('Gender == "female"') \
    .groupby(['Year', 'Country']) \
    .agg({'Salary': 'mean' }) \
    .reset_index() \
    .rename(columns={'Salary': 'SalaryWomen'})

salary_gap_df = pd.merge(salary_men_df, salary_women_df, on=['Year', 'Country']).dropna()
salary_gap_df['SalaryGap'] = abs(salary_gap_df['SalaryMen'] - salary_gap_df['SalaryWomen'])
salary_gap_df['SalaryGapPercent'] = (salary_gap_df['SalaryMen'] - salary_gap_df['SalaryWomen']) / salary_gap_df['SalaryMen'] * 100

salary_gap_df.head()

In [ ]:
# Calculate the mean salary gap for all countries for all genders.
mean_salary_men_df = countries_df.query('Gender == "male"') \
    .groupby(['Year', 'Country']) \
    .agg({'Salary': 'mean' }) \
    .reset_index() \
    .rename(columns={'Salary': 'SalaryMen'})

mean_salary_women_df = countries_df.query('Gender == "female"') \
    .groupby(['Year', 'Country']) \
    .agg({'Salary': 'mean' }) \
    .reset_index() \
    .rename(columns={'Salary': 'SalaryWomen'})

mean_salary_gap_df = pd.merge(mean_salary_men_df, mean_salary_women_df, on=['Year', 'Country']).dropna()
mean_salary_gap_df['SalaryGap'] = abs(mean_salary_gap_df['SalaryMen'] - mean_salary_gap_df['SalaryWomen'])
mean_salary_gap_df['SalaryGapPercent'] = (mean_salary_gap_df['SalaryMen'] - mean_salary_gap_df['SalaryWomen']) / mean_salary_gap_df['SalaryMen'] * 100

mean_salary_gap = mean_salary_gap_df.groupby('Year', as_index=False)['SalaryGapPercent'].mean()

mean_salary_gap

In [14]:
color_alpha = 0.7

fig = px.line(
    salary_gap_df,
    x='Year',
    y='SalaryGapPercent',
    color='Country',
    hover_name='Country',
    title='Top-10 Country Salary Gap Between Men and Women<br><sup>Generally decreasing, but still male-favoured</sup>',
    custom_data=['Country'],
    color_discrete_sequence=[
        f'rgba(233, 146, 119, {color_alpha})',
        f'rgba(148, 233, 119, {color_alpha})',
        f'rgba(119, 127, 233, {color_alpha})',
        f'rgba(218, 119, 233, {color_alpha})',
        f'rgba(218, 191, 162, {color_alpha})',
        f'rgba(207, 150, 150, {color_alpha})',
        f'rgba(167, 199, 213, {color_alpha})',
        f'rgba(233, 119, 186, {color_alpha})',
        f'rgba(154, 162, 221, {color_alpha})',
        f'rgba(154, 221, 209, {color_alpha})',
    ],
    range_y=[0, 48],
    width=790,
)

fig.update_layout(yaxis_title='Salary gap in percentage')

# Add male/female ratio in the legend names
total_male_participants_per_country_df = countries_df.query('Gender == "male"') \
    .groupby('Country', as_index=False) \
    .size() \
    .rename(columns={'size': 'TotalMale'})

total_female_participants_per_country_df = countries_df.query('Gender == "female"') \
    .groupby('Country', as_index=False) \
    .size() \
    .rename(columns={'size': 'TotalFemale'})

total_participants_per_country_df = pd.merge(total_male_participants_per_country_df, total_female_participants_per_country_df)
total_participants_per_country_df['MaleRatioPerFemale'] = total_participants_per_country_df['TotalMale'] / total_participants_per_country_df['TotalFemale']

# Add female/male ratio for each legend name.

# Example: {'oldname': 'newname'}
legend_names = {}
for _, row in total_participants_per_country_df.iterrows():
    legend_names[row['Country']] = f"{row['Country']} (1/{int(row['MaleRatioPerFemale'])})"

fig.for_each_trace(lambda t: t.update(name=legend_names[t.name], 
                                      legendgroup=legend_names[t.name],
                                      hovertemplate='<b>' + t.name + '</b><br>Year: %{x:d} </br>Salary gap: %{y:d}%<extra></extra>'))

# Plot the mean salary scatters
mean_salary_line_fig = px.line(mean_salary_gap,
                               x='Year',
                               y='SalaryGapPercent',
                               markers=True)
for trace in mean_salary_line_fig.data:
    trace.hovertemplate = '<b>' + trace.name + '</b><br>Year: %{x:d} </br>Mean salary gap: %{y:d}%<extra></extra>'

fig.add_traces(list(mean_salary_line_fig.select_traces()))
fig.add_annotation(x=2019.7, y=17.75,
                   showarrow=False,
                   xanchor='center',
                   yanchor='bottom',
                   textangle=-16,
                   text='mean salary gap')

# Add caption
fig.add_annotation(x=-0.11, y=-0.35,
                   xref='paper', yref='paper',
                   showarrow=False,
                   xanchor='left', yanchor='bottom',
                   align='left',
                   text='Salary gaps across ten of the countries with the most survey respondents.<br>Positive percentages indicates a male-favoured salary gap.')

fig.update_layout(
    legend={
        'title': 'Country with<br>female/male ratio<br>'
    },
    margin={'b': 130, 't': 100, 'l': 90}
)

fig.show()